# Climate Change Sonification
## Exploratory Data Analysis

In [8]:
import os
import numpy as np
from netCDF4 import Dataset

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from matplotlib import animation 
from IPython.display import HTML
from matplotlib import rcParams
rcParams["font.family"] = "Times New Roman"

In [9]:
PATH = "data/"
files = sorted([ f for f in os.listdir(path=PATH) if f.endswith(".nc4") ])
co2_data = [ Dataset(PATH + f).variables["XCO2"][:] for f in files ]

In [10]:
vmin = min([ d.min() for d in co2_data ])
vmax = max([ d.max() for d in co2_data ])

In [12]:
%matplotlib notebook

# set up plotting area
fig, ax = plt.subplots()
ax.tick_params(
    left=False,
    bottom=False,
    labelleft=False,
    labelbottom=False
)

# colorbar
ax_divider = make_axes_locatable(ax)
cax = ax_divider.append_axes("right", size="5%", pad="2%")
cmap = mpl.cm.inferno
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
cb = mpl.colorbar.ColorbarBase(
    cax, 
    cmap=cmap,
    norm=norm,
    orientation="vertical"
)

def plot_img(data):
    """Plot CO2 values"""
    data = np.flip(data, axis=0)
    ax.imshow(
        data,
        cmap=cmap, 
        vmin=vmin, 
        vmax=vmax,
        animated=True
    )

# for blitting (optimized raster image animation)
im = plot_img(co2_data[0][0,:,:])

def update(i):  
    im = plot_img(co2_data[i][0,:,:])
    
    # axis title
    date = files[i].split("_")[4]
    ax.set_title(date[-2:] + " - " + date[:-2])

# animate
ani = animation.FuncAnimation(
    fig, 
    update, 
    frames=len(files), 
    interval=500, 
    repeat=True, 
    repeat_delay=1000,
    blit=True
)

# save and display
plt.tight_layout()
ani.save('img/animation.gif', writer='imagemagick')
HTML(ani.to_html5_video())

<IPython.core.display.Javascript object>